# Prepare BOLD5000 data for input into a deep learning model

This notebook takes the BOLD5000 dataset and prepares it for use in a deep learning model. Since the dataset is the direct output of fmriprep, we still need to perform a few steps before it is ready to train on.

First, we must regress out nuisance signals. In fMRI analysis, many nuisance signals are gathering from the processing pipeline and are linearly regressed or detrended from the whole brain timeseries. Some commonly used nuisance signals include motion parameters (rigid body motion involves three translation, three rotation parameters), average CSF signal, average white matter signal, and all their derivatives. The regression of the global signal - or the mean signal across all brain voxels - is highly contended in the fMRI field. Many consider it to be physiological noise, others consider it to contain important neural information. For our purposes, we will not regress out global signal. Our goal is to decode visual stimuli features from fMRI timeseries of visual areas of the brain. A recent study on the global signal showed that the visual cortex contains much of variance in the global signal topography [1]. For this reason, we will only regress out six motion parameters, two biological, and their derivatives. We can pull these signals from one of the fmriprep files.

Second, we must extract the timeseries data of brain regions that are involved in visual processing. We will use the visual regions defined in the HCP MMP 1.0 atlas. We will take these binary masks and mutiply them to each subject's fMRI data to extract the information we need.

Finally, we need to label each part of the timeseries data with the image it corresponds to and package the resulting data into neat X and Y matrices.

[1] Li, J., Bolt, T., Bzdok, D., Nomi, J. S., Yeo, B. T. T., Spreng, R. N., & Uddin, L. Q. (2019). Topography and behavioral relevance of the global signal in the human brain. Scientific Reports, 9(1), 1–10. https://doi.org/10.1038/s41598-019-50750-8

In [2]:
import numpy as np
import os
import sys
from glob import glob
import pandas as pd
import subprocess
from scipy.io import savemat

"""
Takes a subject dataframe loaded from the regressors.tsv file and extracts FD.
Performs motion scrubbing by marking all volumes with FD greater than threshold with 1.
Additionally marks one volume before and two after. Converts to a sparse matrix where
each column has one frame that is censored.
"""
def get_censored_frames(subj_df, threshold):
    # Threshold should be bounded between 0 and 1
    if threshold < 0 or threshold > 1:
        raise ValueError('Threshold should be bounded between 0 and 1.')
    # Extract FD column
    fd = subj_df['FramewiseDisplacement'].values
    fd = np.nan_to_num(fd)
    # Create censor vector
    censor = [0 if m <= threshold else 1 for m in fd]
    # Censor one back, two forward
    censor_fixed = np.zeros_like(censor)
    for ind,c in enumerate(censor):
        if c == 1:
            try:
                censor_fixed[ind-1:ind+3] = 1
            except IndexError:
                censor_fixed[ind-1:] = 1
    # Convert to sparse matrix
    censor_feat = np.zeros((censor_fixed.shape[0], np.count_nonzero(censor_fixed)))
    col = 0
    for ind,c in enumerate(censor_fixed):
        if c == 1:
            censor_feat[ind,col] = 1
            col +=1

    return censor_feat, censor_fixed

"""
Takes a subject dataframe loaded from the regressors.tsv file and extracts relevant regressors (list)
"""
def get_regressors(subj_df, regressors):
    # Should be of dim TRs x # regressors
    regress_mat = np.array([subj_df[regressor].values for regressor in regressors]).T
    # Calculate derivatives manually
    deriv = np.diff(regress_mat,axis=0)
    deriv = np.insert(deriv, 0, regress_mat[0], axis = 0)
    final = np.hstack((regress_mat,deriv))
    return final

"""
Returns subject directories from fmriprep directory
"""
def get_subj_dirs(fmriprep_dir):
    subj_dirs = [f for f in os.listdir(fmri_dir) if os.path.isdir(os.path.join(fmri_dir, f)) and 'sub' in f]
    return subj_dirs

### Registration and nuisance signal regression

The subjects' fMRI data are in their native scanner space. It will be difficult to extract the same regions if there are not all aligned to the same standard space. We will do this using FLIRT to perform an affine transformation to the MNI standard space before we regress.

In [2]:
"""
PARAMETERS - change these if you wish

fmri_dir: where the fmriprep output data lives, should contain subject folders
nuisance_dir: where the confounds_regressors.tsv files are located, should contain subject folders
regressors_dir: an output directory to hold the nuisance regressors text files
preproc_dir: an output directory for the fully processed subject data

"""
fd_threshold = 0.5 # Threshold of FD for censoring a frame
# All the nuisance regressors we wish to remove. Do not include derivatives, these are calculated manually
regressors = ['CSF', 'WhiteMatter','X','Y','Z','RotX','RotY','RotZ'] 
# Set directories
fmri_dir, nuisance_dir, regressors_dir, preproc_dir, standard = ['dataset/ds001499-download/',
                                                       'dataset/ds001499-download/derivatives/fmriprep/',
                                                       'dataset/regressors/',
                                                       'dataset/preprocessed/',
                                                       '/usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz']

In [5]:
# Get all subject directories
subj_dirs = get_subj_dirs(fmri_dir)

# Loop through each subjects and get regressors, perform scrubbing
for subj in sorted(subj_dirs):
    print('Processing %s' % subj)
    # Absolute path of current subject
    subj_dir_abs = os.path.join(fmri_dir, subj)
    sess_dirs = sorted([f for f in os.listdir(subj_dir_abs) if os.path.isdir(os.path.join(subj_dir_abs, f)) and 'ses-' in f])
    if not sess_dirs: # If there are not multiple sessions, then set to list of empty string to iterate only once in for loop
        sess_dirs = ['']
    for sessnum,sess in enumerate(sess_dirs):
        print('\tSession %d out of %d' % ((sessnum + 1), len(sess_dirs)))
        # Absolute path of current session
        sess_dir_abs = os.path.join(subj_dir_abs, sess)
        conf_sess_dir_abs = os.path.join(nuisance_dir, subj, 'ses-' + str(sessnum+1).zfill(2))
        bold_files = sorted(glob(sess_dir_abs + '/func/*task-5000scenes*bold.nii.gz'))
        confound_files = sorted(glob(conf_sess_dir_abs + '/func/*task-5000scenes*confounds*.tsv'))
        # For multiple runs
        for runnum, (bold, confound) in enumerate(zip(bold_files, confound_files)):
            print('\t\tRun %d out of %d' % ((runnum + 1), len(bold_files)))
            df = pd.read_csv(confound, sep='\t')
            censor_mat, censor_frames = get_censored_frames(df, fd_threshold)
            regress_mat = get_regressors(df, regressors)
            nuisance_mat = np.hstack((censor_mat,regress_mat))
            prefix = os.path.join(regressors_dir, subj + '_ses-' + str(sessnum+1).zfill(2) + '_run-' + str(runnum+1).zfill(2) + '_')
            outfile = prefix + 'censored.txt'
            np.savetxt(outfile, censor_frames)
            outfile = prefix + 'nuisance_regressors.txt'
            np.savetxt(outfile, nuisance_mat)
            # Perform registration
            bold_reg = bold[:-7] + '_registered.nii.gz'
            outmat = os.path.join(sess_dir_abs,'func',subj + '_ses-' + str(sessnum+1).zfill(2) + '_run-' + str(runnum+1).zfill(2) + '_native2template.mat')
            cmd = 'flirt -in ' + bold + ' -ref ' + standard + ' -omat ' + outmat
            subprocess.call(cmd, shell = True)
            cmd = 'flirt -in ' + bold + ' -ref ' + standard + ' -out ' + bold_reg + ' -init ' + outmat + ' -applyxfm -v'
            subprocess.call(cmd, shell = True)
            # Use AFNI to perform regression
            outfile = outfile[:-3] + 'mat'
            savemat(outfile, {'nuisance_regressors': regress_mat})
            subprocess.call('python2 /home/ubuntu/abin/read_matlab_files.py -infiles ' + outfile + ' -prefix ' + prefix[:-1] + ' -overwrite', shell = True)
            design = glob(prefix[:-1] + '*.1D')[0]
            prefix = os.path.join(preproc_dir, subj + '_ses-' + str(sessnum+1).zfill(2) + '_run-' + str(runnum+1).zfill(2) + '_')
            outfile = prefix + 'preproc.nii.gz'
            subprocess.call('3dTproject -input ' + bold_reg + ' -prefix ' + outfile + ' -ort ' + design + ' -polort 2 -passband 0.009 0.1 -blur 6 -quiet -overwrite', shell = True)

Processing sub-CSI1
	Session 1 out of 16
		Run 1 out of 10
		Run 2 out of 10
		Run 3 out of 10
		Run 4 out of 10
		Run 5 out of 10
		Run 6 out of 10
		Run 7 out of 10
		Run 8 out of 10
		Run 9 out of 10
		Run 10 out of 10
	Session 2 out of 16
		Run 1 out of 10
		Run 2 out of 10
		Run 3 out of 10
		Run 4 out of 10
		Run 5 out of 10
		Run 6 out of 10
		Run 7 out of 10
		Run 8 out of 10
		Run 9 out of 10
		Run 10 out of 10
	Session 3 out of 16
		Run 1 out of 10
		Run 2 out of 10
		Run 3 out of 10
		Run 4 out of 10
		Run 5 out of 10
		Run 6 out of 10
		Run 7 out of 10
		Run 8 out of 10
		Run 9 out of 10
		Run 10 out of 10
	Session 4 out of 16
		Run 1 out of 9
		Run 2 out of 9
		Run 3 out of 9
		Run 4 out of 9
		Run 5 out of 9
		Run 6 out of 9
		Run 7 out of 9
		Run 8 out of 9
		Run 9 out of 9
	Session 5 out of 16
		Run 1 out of 10
		Run 2 out of 10
		Run 3 out of 10
		Run 4 out of 10
		Run 5 out of 10
		Run 6 out of 10
		Run 7 out of 10
		Run 8 out of 10
		Run 9 out of 10
		Run 10 out of 1

		Run 2 out of 9
		Run 3 out of 9
		Run 4 out of 9
		Run 5 out of 9
		Run 6 out of 9
		Run 7 out of 9
		Run 8 out of 9
		Run 9 out of 9
	Session 15 out of 16
		Run 1 out of 10
		Run 2 out of 10
		Run 3 out of 10
		Run 4 out of 10
		Run 5 out of 10
		Run 6 out of 10
		Run 7 out of 10
		Run 8 out of 10
		Run 9 out of 10
		Run 10 out of 10
	Session 16 out of 16
Processing sub-CSI4
	Session 1 out of 10
		Run 1 out of 10
		Run 2 out of 10
		Run 3 out of 10
		Run 4 out of 10
		Run 5 out of 10
		Run 6 out of 10
		Run 7 out of 10
		Run 8 out of 10
		Run 9 out of 10
		Run 10 out of 10
	Session 2 out of 10
		Run 1 out of 9
		Run 2 out of 9
		Run 3 out of 9
		Run 4 out of 9
		Run 5 out of 9
		Run 6 out of 9
		Run 7 out of 9
		Run 8 out of 9
		Run 9 out of 9
	Session 3 out of 10
		Run 1 out of 9
		Run 2 out of 9
		Run 3 out of 9
		Run 4 out of 9
		Run 5 out of 9
		Run 6 out of 9
		Run 7 out of 9
		Run 8 out of 9
		Run 9 out of 9
	Session 4 out of 10
		Run 1 out of 10
		Run 2 out of 10
		Run 3 out 

### ROI masking and train data preparation

Now that we have the fMRI data fully preprocessed, we will need to extract the ROI timeseries of all the visual region masks that BOLD5000 kindly provided, and match the correct labels from the events file. The result will be an X and Y for each ROI mask. The X will be of shape (samples, timepoints, features) and Y will be of shape (samples, classes). Since each stimuli is presented for about 1 seconds with 9 seconds in between each, we will take 10 second windows for each class label. With a TR of 2 seconds, each sample will have 5 TRs of all the voxels in that ROI for each corresponding class label. The datasets will be concatenated for all runs, sessions, and subjects. Thus, the resulting data will be saved in the output directory specified (data_dir), one X and Y for each ROI.

Again, stick with Python 2 for the sake of the AFNI commands used. Also, be sure to specify the directories at the top of the following cell.

This will take only a few hours to run.

In [3]:
import pandas as pd
from collections import defaultdict
import nibabel as nib

"""
DIRECTORIES - BE SURE TO SET THESE!!

roi_dir: where the roi_masks are located, should contain subject folders
preproc_dir: where the fully processed data is stored
events_dir: where the event files are held, should contain subject folders.
            This is probably the same folder as the original dataset/fmriprep folder.
data_dir: the output of where you want the training data to be saved
mask_dir: the output of where you want your resampled roi masks to be
            
"""
preproc_dir, events_dir, data_dir, mask_dir = ['dataset/preprocessed/',
                                            'dataset/ds001499-download/',
                                            'dataset/traindata/',
                                            'dataset/masks/']

# Dicts holding training set and labels for each mask
X = []
Y = []

# Holds subject, session, run information for each sample
sample2image2session = pd.DataFrame()

In [4]:
# Get masks in mask directory
mask_files = [os.path.join(mask_dir,f) for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir,f)) and 'nii.gz' in f]

# Walk through ROI mask directory
for mask_file in mask_files:
    preproc_files = glob(preproc_dir + '*_preproc.nii.gz')
    for pnum, preproc in enumerate(preproc_files):
        print('Preprocessed file %d out of %d' % ((pnum + 1), len(preproc_files)))
        items = preproc.split('_')
        ses = items[-3]
        run = items[-2]
        subname = items[0].split('/')[-1]
        event_file = glob(os.path.join(events_dir,subname,ses,'func','*' + run + '_events.tsv'))[0]
        # Load events and image
        events = pd.read_csv(event_file, sep = '\t')
        img = nib.load(preproc).get_fdata()
        mask = nib.load(mask_file).get_fdata().astype(bool)
        # Apply mask
        roi = img[mask] # Shape: voxels x TRs
        # Get relevant time intervals and labels from events file
        for index, row in events.iterrows():
            # Beginning TR of trial
            start = int(round(row['onset']) / 2)
            # Ending TR of trial, start + 10 sec, or 5 TRs
            end = start + 5
            x = roi[:,start:end].T
            X.append(x) # Big X should be of shape (samples, timepoints, features)
            Y.append(row['ImgName'])
            sample2image2session = sample2image2session.append({'SampleIndex': len(X) - 1, 
                                                                'ImgName': row['ImgName'], 
                                                                'Subject': subname,
                                                                'Session': int(ses[-2:]),
                                                                'Run': int(run[-2:])}, ignore_index = True)
        # Save last ten TRs as no stimulus, if enough data is left
        if roi.shape[1] - end >= 5:
            x = roi[:,end:end+5].T
            X.append(x)
            Y.append('none')
            sample2image2session = sample2image2session.append({'SampleIndex': len(X) - 1, 
                                                                'ImgName': 'none', 
                                                                'Subject': subname,
                                                                'Session': int(ses[-2:]),
                                                                'Run': int(run[-2:])}, ignore_index = True)

Preprocessed file 1 out of 510
Preprocessed file 2 out of 510
Preprocessed file 3 out of 510
Preprocessed file 4 out of 510
Preprocessed file 5 out of 510
Preprocessed file 6 out of 510
Preprocessed file 7 out of 510
Preprocessed file 8 out of 510
Preprocessed file 9 out of 510
Preprocessed file 10 out of 510
Preprocessed file 11 out of 510
Preprocessed file 12 out of 510
Preprocessed file 13 out of 510
Preprocessed file 14 out of 510
Preprocessed file 15 out of 510
Preprocessed file 16 out of 510
Preprocessed file 17 out of 510
Preprocessed file 18 out of 510
Preprocessed file 19 out of 510
Preprocessed file 20 out of 510
Preprocessed file 21 out of 510
Preprocessed file 22 out of 510
Preprocessed file 23 out of 510
Preprocessed file 24 out of 510
Preprocessed file 25 out of 510
Preprocessed file 26 out of 510
Preprocessed file 27 out of 510
Preprocessed file 28 out of 510
Preprocessed file 29 out of 510
Preprocessed file 30 out of 510
Preprocessed file 31 out of 510
Preprocessed file

Preprocessed file 253 out of 510
Preprocessed file 254 out of 510
Preprocessed file 255 out of 510
Preprocessed file 256 out of 510
Preprocessed file 257 out of 510
Preprocessed file 258 out of 510
Preprocessed file 259 out of 510
Preprocessed file 260 out of 510
Preprocessed file 261 out of 510
Preprocessed file 262 out of 510
Preprocessed file 263 out of 510
Preprocessed file 264 out of 510
Preprocessed file 265 out of 510
Preprocessed file 266 out of 510
Preprocessed file 267 out of 510
Preprocessed file 268 out of 510
Preprocessed file 269 out of 510
Preprocessed file 270 out of 510
Preprocessed file 271 out of 510
Preprocessed file 272 out of 510
Preprocessed file 273 out of 510
Preprocessed file 274 out of 510
Preprocessed file 275 out of 510
Preprocessed file 276 out of 510
Preprocessed file 277 out of 510
Preprocessed file 278 out of 510
Preprocessed file 279 out of 510
Preprocessed file 280 out of 510
Preprocessed file 281 out of 510
Preprocessed file 282 out of 510
Preprocess

Preprocessed file 502 out of 510
Preprocessed file 503 out of 510
Preprocessed file 504 out of 510
Preprocessed file 505 out of 510
Preprocessed file 506 out of 510
Preprocessed file 507 out of 510
Preprocessed file 508 out of 510
Preprocessed file 509 out of 510
Preprocessed file 510 out of 510


#### Save the data

In [6]:
import pickle

with open(data_dir + 'X_raw.p', 'wb') as f:
    pickle.dump(X, f)
    
with open(data_dir + 'Y_raw.p', 'wb') as f:
    pickle.dump(Y, f)

with open(data_dir + 'sample2session.p', 'wb') as f:
    pickle.dump(sample2image2session, f)

# Process data from Shen et al. 2019, "Deep image reconstruction from human brain activity"

In [4]:
"""
PARAMETERS - change these if you wish

fmri_dir: where the fmriprep output data lives, should contain subject folders
nuisance_dir: where the confounds_regressors.tsv files are located, should contain subject folders
regressors_dir: an output directory to hold the nuisance regressors text files
preproc_dir: an output directory for the fully processed subject data

"""
fd_threshold = 0.5 # Threshold of FD for censoring a frame
# All the nuisance regressors we wish to remove. Do not include derivatives, these are calculated manually
regressors = ['CSF', 'WhiteMatter','X','Y','Z','RotX','RotY','RotZ'] 
# Set directories
fmri_dir, nuisance_dir, regressors_dir, preproc_dir, standard = ['../deepImageRecon/ds001506-download/',
                                                                 '../deepImageRecon/ds001506-download/',
                                                                 '../deepImageRecon/regressors/',
                                                                 '../deepImageRecon/preprocessed/',
                                                                 '/usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz']

In [5]:
# Get all subject directories
subj_dirs = get_subj_dirs(fmri_dir)

# Loop through each subjects and get regressors, perform scrubbing
for subj in sorted(subj_dirs):
    print('Processing %s' % subj)
    # Absolute path of current subject
    subj_dir_abs = os.path.join(fmri_dir, subj)
    sess_dirs = sorted([f for f in os.listdir(subj_dir_abs) if os.path.isdir(os.path.join(subj_dir_abs, f)) and 'ses-' in f and 'NaturalImage' in f])
    if not sess_dirs: # If there are not multiple sessions, then set to list of empty string to iterate only once in for loop
        sess_dirs = ['']
    for sessnum,sess in enumerate(sess_dirs):
        print('\tSession %d out of %d' % ((sessnum + 1), len(sess_dirs)))
        # Absolute path of current session
        sess_dir_abs = os.path.join(subj_dir_abs, sess)
        conf_sess_dir_abs = os.path.join(nuisance_dir, subj, sess)
        bold_files = sorted(glob(sess_dir_abs + '/func/*task-perception*bold.nii.gz'))
        # For multiple runs
        for runnum, bold in enumerate(bold_files):
            print('\t\tRun %d out of %d' % ((runnum + 1), len(bold_files)))
            # Perform registration
            prefix = os.path.join(preproc_dir, subj + '_' + sess + '_run-' + str(runnum+1).zfill(2) + '_')
            outfile = prefix + 'preproc.nii.gz'
            outmat = os.path.join(sess_dir_abs,'func',subj + '_' + sess + '_run-' + str(runnum+1).zfill(2) + '_native2template.mat')
            cmd = 'flirt -in ' + bold + ' -ref ' + standard + ' -omat ' + outmat
            subprocess.call(cmd, shell = True)
            cmd = 'flirt -in ' + bold + ' -ref ' + standard + ' -out ' + outfile + ' -init ' + outmat + ' -applyxfm -v'
            subprocess.call(cmd, shell = True)

Processing sub-01
	Session 1 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 2 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 3 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 4 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 5 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 6 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 7 out of 18
		Run 1 out of 8
		

		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 17 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
	Session 18 out of 18
		Run 1 out of 8
		Run 2 out of 8
		Run 3 out of 8
		Run 4 out of 8
		Run 5 out of 8
		Run 6 out of 8
		Run 7 out of 8
		Run 8 out of 8
